In [153]:
import pandas as pd
import numpy as np
import datetime
import dateutil.parser
from glob import glob
import os

In [ ]:
def make_subdataset(elec, meta, weather_path,weather_file):
    #read tables
    print(weather_file)
    weather = pd.read_csv(weather_path+weather_file,parse_dates=['timestamp'])
    #parse date
#     weather['timestamp'] = weather['timestamp'].apply(dateutil.parser.parse)
    weather['rounded_timestamp'] = weather['timestamp'].apply(cutoff_minute)
    weather = weather.groupby('rounded_timestamp').first()
    print(weather.index)#only the first observation in each hour is taken
    weather = weather['TemperatureC']
    #construct the dataframe to return
    buildings = list(meta[(meta['newweatherfilename'] == weather_file)].index)
    print
    #name of the buildings
    df = pd.DataFrame() #empty dataframe with 3 columns
    print(len(buildings))
    if len(buildings) == 0:
        return df
    else:
        for building in buildings:
            subdf = pd.DataFrame()
            subdf = elec[['timestamp', building]]
            subdf.columns = ['timestamp', 'electricity']
            if len(subdf['timestamp'])> 8757:
                subdf = subdf.iloc[:8757]
            subdf['building_name'] = building
            subdf['weather_file'] = weather_file
            subdf['industry'] = meta.loc[building,'industry']
            subdf['area'] = meta.loc[building, 'sqm']
            subdf['primary_space_usage'] = meta.loc[building, 'primaryspaceuse_abbrev']
            df = pd.concat([df, subdf], axis=0, ignore_index=True)
    df['rounded_timestamp'] = df['timestamp'].apply(cutoff_minute)
    df = df.groupby('rounded_timestamp').first()
    print('END SUBDF DF:',df.shape)
    df = df.join(weather.copy(), how='inner', lsuffix='elec', rsuffix='weather') #join temperature data from weather table
    return df

def add_onehot_dates(df):
    df = add_time_columns(df)
    df = pd.concat([df,pd.get_dummies(df['month'], prefix='month')],axis=1)
    df = pd.concat([df,pd.get_dummies(df['date'], prefix='date')],axis=1)
    df = pd.concat([df,pd.get_dummies(df['hour'], prefix='hour')],axis=1)
    df = pd.concat([df,pd.get_dummies(df['weekday'], prefix='weekday')],axis=1)
    return df

def cutoff_minute(dt):
    '''
    INPUT
    a datetime object has year, month, day, hour, and minute
    OUTPUT
    a datetime object has year, month, day, and hour
    '''
    year = dt.year
    month = dt.month
    day = dt.day
    hour = dt.hour
    return datetime.datetime(year, month, day, hour)

In [217]:
def make_dataset(elec, meta, weather_path):
    meta = meta.set_index('uid')
    try:
        elec['timestamp'] = elec['timestamp'].apply(dateutil.parser.parse)
    except:
        pass
    elec['timestamp'] = elec['timestamp'].apply(cutoff_minute)
    weather_list = os.listdir(weather_path)
    weather_list = [w.split('/')[-1] for w in weather_list]
#     meta_files_list = [formeta['newweatherfilename']]
    df = pd.DataFrame()
    for weather_file in weather_list:
        subdf = make_subdataset(elec, meta, weather_path,weather_file)
        print('Dataframe DF:',df.shape,'SubDF:',subdf.shape)
        df = pd.concat([df, subdf], axis=0, ignore_index=True)
        print(df.shape)
    df = pd.concat([df,pd.get_dummies(df['primary_space_usage'], prefix='psu')],axis=1)
    df = add_onehot_dates(df)
    df = df.set_index(['building_name','timestamp']).unstack(1)
    return df

In [214]:
def add_time_columns(df):
    year_list = [y.year for y in df['timestamp']]
    month_list = [y.month for y in df['timestamp']]
    date_list = [y.day for y in df['timestamp']]
    hour_list = [y.hour for y in df['timestamp']]
    weekday_list = [y.weekday() for y in df['timestamp']]
    df['year'] = year_list
    df['month'] = month_list
    df['date'] = date_list
    df['hour'] = hour_list
    df['weekday'] = weekday_list
    return df

In [ ]:
def add_weather_file(df,weather_file):
    weather_list = [weather_file for y in df['timestamp']]
    df['weather_file'] 

In [210]:
meta = pd.read_csv('/Users/kategroschner/Desktop/the-building-data-genome-project-master/data/raw/meta_open.csv',parse_dates=['dataend','datastart'])
elec = pd.read_csv('/Users/kategroschner/Desktop/the-building-data-genome-project-master/data/processed/temp_open_utc_complete.csv',parse_dates=['timestamp'])

In [28]:
weather_path = '/Users/kategroschner/Desktop/the-building-data-genome-project-master/data/external/weather/'

In [ ]:
df = make_dataset(elec, meta, weather_path)

In [73]:
df.head()

,TemperatureC,area,building_name,date,date_1,date_10,date_11,date_12,date_13,date_14,...,weather_file,weekday,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,year
0,-11.6,7740.770863,Office_Dawn,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,weather13.csv,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2010.0
1,-11.6,10319.386530,Office_Dorian,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,weather13.csv,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2010.0
2,-11.6,14279.191100,UnivLab_Dianna,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,weather13.csv,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2010.0
3,-11.5,7740.770863,Office_Dawn,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,weather13.csv,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2010.0
4,-11.5,10319.386530,Office_Dorian,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,weather13.csv,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2010.0


In [78]:
df['weather_file'].drop_duplicates()

0    weather13.csv
Name: weather_file, dtype: object

In [88]:
q = df[df['building_name']=='Office_Dawn']

In [89]:
len(q)

8725

In [74]:
df.shape

(26175, 89)

In [152]:
meta.head()

,uid,dataend,datastart,energystarscore,heatingtype,industry,mainheatingtype,numberoffloors,occupants,primaryspaceusage,rating,sqft,sqm,subindustry,timezone,yearbuilt,nickname,primaryspaceuse_abbrev,newweatherfilename
0,PrimClass_Everett,2012-12-31 23:00:00,2012-01-01,NaN,NaN,Education,NaN,NaN,NaN,Primary/Secondary Classroom,NaN,105530.0,9804.053590,Primary/Secondary School,America/New_York,NaN,Everett,PrimClass,weather12.csv
1,UnivClass_Clifford,2015-12-31 23:00:00,2015-01-01,NaN,NaN,Education,NaN,NaN,NaN,College Classroom,NaN,56969.0,5292.591007,College/University,America/New_York,1967,Clifford,UnivClass,weather2.csv
2,Office_Elizabeth,2012-12-31 23:00:00,2012-01-01,NaN,NaN,Commercial Property,NaN,NaN,NaN,Office,NaN,294651.0,27373.961850,Commercial Real Estate,America/Los_Angeles,NaN,Elizabeth,Office,weather22.csv
3,Office_Ellie,2012-12-31 23:00:00,2012-01-01,NaN,NaN,Commercial Property,NaN,NaN,NaN,Office,NaN,496517.0,46127.918850,Bank/Financial Services,America/Los_Angeles,NaN,Ellie,Office,weather28.csv
4,PrimClass_Elisabeth,2012-12-31 23:00:00,2012-01-01,NaN,NaN,Education,NaN,NaN,NaN,Primary/Secondary Classroom,NaN,233062.0,21652.158990,Primary/Secondary School,America/New_York,NaN,Elisabeth,PrimClass,weather23.csv


In [202]:
def check_weather_file(meta,weather_path,weather_file):
    weather = pd.read_csv(weather_path+weather_file,parse_dates=['timestamp'])
    m = meta.copy()
    m = m.set_index('newweatherfilename')
    end_list = []
    start_list = []
    for idx,w in enumerate(m.iterrows()):
        print(w[1]['dataend'])
        print(w[1]['datastart'])
        if w[1]['dataend'] not in weather['timestamp']:
            print(w[1]['dataend'])
            end_list.append([idx,w[1]['dataend']])
        elif w[1]['datastart'] not in weather['timestamp']:
            start_list.append([idx,w[1]['dataend']])
    return end_list, start_list

In [204]:
end_list

[[0, Timestamp('2012-12-31 23:00:00')],
 [1, Timestamp('2015-12-31 23:00:00')],
 [2, Timestamp('2012-12-31 23:00:00')],
 [3, Timestamp('2012-12-31 23:00:00')],
 [4, Timestamp('2012-12-31 23:00:00')],
 [5, Timestamp('2015-12-31 23:00:00')],
 [6, Timestamp('2015-11-30 23:00:00')],
 [7, Timestamp('2015-11-30 23:00:00')],
 [8, Timestamp('2015-11-30 23:00:00')],
 [9, Timestamp('2012-12-31 23:00:00')],
 [10, Timestamp('2012-12-31 23:00:00')],
 [11, Timestamp('2012-12-31 23:00:00')],
 [12, Timestamp('2012-12-31 23:00:00')],
 [13, Timestamp('2012-12-31 23:00:00')],
 [14, Timestamp('2012-12-31 23:00:00')],
 [15, Timestamp('2012-12-31 23:00:00')],
 [16, Timestamp('2012-12-31 23:00:00')],
 [17, Timestamp('2012-12-31 23:00:00')],
 [18, Timestamp('2012-12-31 23:00:00')],
 [19, Timestamp('2012-12-31 23:00:00')],
 [20, Timestamp('2012-12-31 23:00:00')],
 [21, Timestamp('2012-12-31 23:00:00')],
 [22, Timestamp('2012-12-31 23:00:00')],
 [23, Timestamp('2012-12-31 23:00:00')],
 [24, Timestamp('2012-12-3

In [203]:
end_list, start_list = check_weather_file(meta,weather_path,'weather00.csv')

2012-12-31 23:00:00
2012-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-11-30 23:00:00
2014-01-12 00:00:00
2015-11-30 23:00:00
2014-01-12 00:00:00
2015-11-30 23:00:00
2014-01-12 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00
2012-12-31 23:00:00
2012-01-01 00:00:00


2014-01-12 00:00:00
2015-04-30 23:00:00
2014-01-05 00:00:00
2015-04-30 23:00:00
2014-01-05 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-11-30 23:00:00
2014-01-12 00:00:00
2015-11-30 23:00:00
2014-01-12 00:00:00
2015-04-30 23:00:00
2014-01-05 00:00:00
2015-11-30 23:00:00
2014-01-12 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-10-31 23:00:00
2014-01-11 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-04-30 23:00:00
2014-01-05 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-10-31 23:00:00
2014-01-11 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-10-31 23:00:00
2014-01-11 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00
2015-01-01 00:00:00
2015-12-31 23:00:00


In [227]:
df2 = make_dataset(elec, meta, weather_path)

weather0.csv
DatetimeIndex(['2015-01-01 00:00:00', '2015-01-01 01:00:00',
               '2015-01-01 02:00:00', '2015-01-01 03:00:00',
               '2015-01-01 04:00:00', '2015-01-01 05:00:00',
               '2015-01-01 06:00:00', '2015-01-01 07:00:00',
               '2015-01-01 08:00:00', '2015-01-01 09:00:00',
               ...
               '2015-12-31 14:00:00', '2015-12-31 15:00:00',
               '2015-12-31 16:00:00', '2015-12-31 17:00:00',
               '2015-12-31 18:00:00', '2015-12-31 19:00:00',
               '2015-12-31 20:00:00', '2015-12-31 21:00:00',
               '2015-12-31 22:00:00', '2015-12-31 23:00:00'],
              dtype='datetime64[ns]', name='rounded_timestamp', length=8740, freq=None)
96
END SUBDF DF: (840672, 7)
Dataframe DF: (0, 0) SubDF: (0, 8)
(0, 8)
weather1.csv
DatetimeIndex(['2014-12-01 00:00:00', '2014-12-01 01:00:00',
               '2014-12-01 02:00:00', '2014-12-01 03:00:00',
               '2014-12-01 04:00:00', '2014-12-01 05:00:00',
  

DatetimeIndex(['2012-01-01 00:00:00', '2012-01-01 01:00:00',
               '2012-01-01 03:00:00', '2012-01-01 04:00:00',
               '2012-01-01 05:00:00', '2012-01-01 06:00:00',
               '2012-01-01 07:00:00', '2012-01-01 08:00:00',
               '2012-01-01 09:00:00', '2012-01-01 10:00:00',
               ...
               '2012-12-31 14:00:00', '2012-12-31 15:00:00',
               '2012-12-31 16:00:00', '2012-12-31 17:00:00',
               '2012-12-31 18:00:00', '2012-12-31 19:00:00',
               '2012-12-31 20:00:00', '2012-12-31 21:00:00',
               '2012-12-31 22:00:00', '2012-12-31 23:00:00'],
              dtype='datetime64[ns]', name='rounded_timestamp', length=8547, freq=None)
1
END SUBDF DF: (8757, 7)
Dataframe DF: (0, 8) SubDF: (0, 8)
(0, 8)
weather2.csv
DatetimeIndex(['2015-01-01 00:00:00', '2015-01-01 01:00:00',
               '2015-01-01 02:00:00', '2015-01-01 03:00:00',
               '2015-01-01 04:00:00', '2015-01-01 05:00:00',
               '20

KeyboardInterrupt: 

In [216]:
meta['newweatherfilename'].drop_duplicates()

0      weather12.csv
1       weather2.csv
2      weather22.csv
3      weather28.csv
4      weather23.csv
6       weather1.csv
9      weather17.csv
10     weather14.csv
11     weather35.csv
13     weather18.csv
15     weather26.csv
17     weather24.csv
18     weather29.csv
19     weather15.csv
20     weather19.csv
21     weather11.csv
28     weather38.csv
30     weather16.csv
31     weather10.csv
32      weather8.csv
33     weather21.csv
34     weather32.csv
35     weather20.csv
37     weather36.csv
40     weather37.csv
44      weather4.csv
46     weather27.csv
49      weather5.csv
52     weather25.csv
55     weather31.csv
58     weather30.csv
76      weather3.csv
86      weather7.csv
129     weather0.csv
145     weather6.csv
188     weather9.csv
460    weather13.csv
487    weather33.csv
Name: newweatherfilename, dtype: object

In [230]:
'weather34.csv' in meta['newweatherfilename'].drop_duplicates()

False

In [233]:
np.sort(meta['newweatherfilename'].drop_duplicates())

array(['weather0.csv', 'weather1.csv', 'weather10.csv', 'weather11.csv',
       'weather12.csv', 'weather13.csv', 'weather14.csv', 'weather15.csv',
       'weather16.csv', 'weather17.csv', 'weather18.csv', 'weather19.csv',
       'weather2.csv', 'weather20.csv', 'weather21.csv', 'weather22.csv',
       'weather23.csv', 'weather24.csv', 'weather25.csv', 'weather26.csv',
       'weather27.csv', 'weather28.csv', 'weather29.csv', 'weather3.csv',
       'weather30.csv', 'weather31.csv', 'weather32.csv', 'weather33.csv',
       'weather35.csv', 'weather36.csv', 'weather37.csv', 'weather38.csv',
       'weather4.csv', 'weather5.csv', 'weather6.csv', 'weather7.csv',
       'weather8.csv', 'weather9.csv'], dtype=object)